<a href="https://colab.research.google.com/github/vi-shruti/Kaggle/blob/master/RevenuePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

In [0]:
train = pd.read_csv(train_ship_segmentations_v2.csv)

In [0]:
print(train.head())
print(test.head())